In [28]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

<b> Grab Tweets <b>

In [29]:
def grab_tweets():
    """
    The user supplied these data already split into training and test sets.
    This function downloads and combines them, returning a single data frame.
    """
    url1 = "https://raw.githubusercontent.com/PhilChodrow/PIC16A/master/datasets/Corona_NLP_train.csv"
    url2 = "https://raw.githubusercontent.com/PhilChodrow/PIC16A/master/datasets/Corona_NLP_test.csv"
    df1 = pd.read_csv(url1, encoding='iso-8859-1')
    df2 = pd.read_csv(url2, encoding='iso-8859-1')
    return pd.concat((df1, df2), axis = 0).reset_index().drop("index", axis = 1)
df = grab_tweets()

<b> Term Document Matrix <b>

In [30]:
vec = CountVectorizer(max_df = 0.2, min_df = 50, stop_words = 'english')
count = vec.fit_transform(df['OriginalTweet'])
count_df = pd.DataFrame(count.toarray(), columns = sorted(vec.vocabulary_.keys()))
df = pd.concat((df, count_df), axis = 1)

train, test = train_test_split(df, test_size = 0.5)

<b> Prepare Predictor and Target Variables<b>

In [31]:
def prepare_variables(data):
    list_of_columns = ['UserName', 'ScreenName', 'Location', 'TweetAt','OriginalTweet', 'Sentiment']
    X = data.drop(list_of_columns, axis = 1)
    y = data['Sentiment'].str.contains("Positive").astype(int)
    return X, y

In [32]:
X_train, y_train = prepare_variables(train)
X_test, y_test = prepare_variables(test)

<b>Fitting the Model with Logistic Regression<b> 

In [33]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/Applications/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [34]:
print("Training score: {}".format(lr.score(X_train, y_train)))
print("Test score: {}".format(lr.score(X_test, y_test)))

Training score: 0.8763625038928683
Test score: 0.8377969570246463


<b> Machine Learned Positive and Negative Words <b>

In [35]:
result_df = pd.DataFrame({"coef":lr.coef_[0], "word":X_train.columns})

In [36]:
#negative words
result_df.sort_values('coef', ascending = True).head(10)

,coef,word
918,-3.203242,hell
493,-3.124087,crisis
1679,-3.007794,scams
2087,-2.992890,war
1063,-2.812648,killing
1876,-2.667854,suffering
1061,-2.650505,kill
516,-2.635546,dangerous
555,-2.597553,died
1872,-2.596547,stupid


In [37]:
#positive words
result_df.sort_values('coef', ascending = False).head(10)

,coef,word
213,3.478464,best
2137,3.258502,won
891,3.170553,hand
794,3.046746,free
2159,2.988199,wow
1799,2.978968,special
1384,2.914942,perfect
900,2.896759,happy
1671,2.894933,save
863,2.884940,great
